In [2]:
!pip install python-dotenv

!pip install crewai
!pip install 'crewai[tools]'

!pip install openai
!pip install langchain
!pip install -U langchain-openai
!pip install exa_py

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-0.1.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_openai-0.1.7-py3-none-any.whl.metadata (2.5 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
Using cached langchain_openai-0.1.7-py3-none-any.whl (34 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 0.21.0
    Uninstalling python-dotenv-0.21.0:
      Successfully uninstalled python-dotenv-0.21.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.9
    Uninstalling langchain-core-0.2.9:
      Successfully uninstalled langchain-core-0.2.9
  Attempting 

In [1]:
import os
from dotenv import load_dotenv

import openai
from langchain_openai import ChatOpenAI

'''
os.environ["SERPER_API_KEY"] 
os.environ["EXA_API_KEY"] 
os.environ["OPENAI_API_KEY"]
'''
load_dotenv()

openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool, SerperDevTool, ScrapeWebsiteTool, EXASearchTool

from exa_py import Exa

# Initialize the tool for internet searching capabilities
search_tool = SerperDevTool()
webscrape_tool = ScrapeWebsiteTool()


@tool("Exa search and get contents")
def search_and_get_contents_tool(question: str) -> str:
    """Tool using Exa's Python SDK to run semantic search and return result highlights."""

    exa = Exa(os.environ['EXA_API_KEY'])

    response = exa.search_and_contents(
        question,
        type="neural",
        use_autoprompt=True,
        num_results=3,
        highlights=True
    )

    parsedResult = ''.join([f'<Title id={idx}>{eachResult.title}</Title><URL id={idx}>{eachResult.url}</URL><Highlight id={idx}>{"".join(eachResult.highlights)}</Highlight>' for (idx, eachResult) in enumerate(response.results)])

    return parsedResult

exa_tools = search_and_get_contents_tool

In [3]:
openAIGPT35_500 = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=500,
    timeout=None,
    max_retries=2,
)

openAIGPT35_3000 = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=3000,
    timeout=None,
    max_retries=2,
)

openAIGPT4 = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=3000,
    timeout=None,
    max_retries=2,
)

In [4]:
manager = Agent(
    role='Manager',
    goal='Manages the distribution of user input and synthesizes expert discussions to create a comprehensive portrayal about a historical figure or influential person.',
    verbose=True,
    backstory=(
        "Highly organized professional with experience in project management and coordination. \
        Holds a degree in a relevant field such as history, sociology, or library science, \
        with additional training in collaborative processes and interdisciplinary research. \
        Skilled in communication and synthesis of diverse information sources."
    ),
    llm = openAIGPT35_3000,
    max_iter = 10,
)

manager_task1 = Task(
    description=(
        "Our team's goal is to research and analyze personality and bibliographical profile about {historical_figure_1} and {historical_figure_2}, "
        "so that we can provide a user experience where the user can have a hypothetical free discussion with "
        "{historical_figure_1} and {historical_figure_2} about a topic they want."
        "Since we are trying to build a person's personality, some examples of important qualities would be"
        "talking or writing habits, personal beliefs, bibliography, time era and trends, etc are important when researching."
        "Plan, divide, and distribute appropriate tasks to subject matter experts by \
        providing specific areas of focus and context for their investigation about {historical_figure_1} and {historical_figure_2} like mentioned."
        "Ensure that each expert understands their task and encourage collaboration and discussion among them. \
        This facilitates a thorough exploration of the historical figure’s personality, context, and impact, \
        leveraging the diverse expertise of the team."
    ),
    expected_output=(
        "Give a list of how task should be distributed among subject experts to analyze and investigate about {historical_figure_1} and {historical_figure_2}."
        "Example output format: "
        "**Goal**"
        "- Our goal is to work together as a team to analyze and investigate about {historical_figure_1} and {historical_figure_2}."
        "***Agent Biographer's Tasks**"
        "- As you are a expert in (what the agent is an expert at and its ), you're goal is to summarize bibliographical information about {historical_figure_1} and {historical_figure_2}."
        "- Task number 1"
        "- ...more tasks for each agents to achieve their goal"
    ),
    agent=manager,
)

In [5]:
biographer = Agent(
    role='Biographer',
    goal='Focuses on compiling a detailed and accurate biography of a historical figure or someone influential',
    verbose=True,
    memory=True,
    backstory=(
        "The biographer  Accomplished author with a background in journalism or literature. \
        Renowned for writing several critically acclaimed biographies, \
        demonstrating expertise in storytelling and interviewing techniques."
    ),
    llm = openAIGPT35_500,
    max_iter = 10,
)

biographer_task_desc = "Gather comprehensive information about the life of the historical figures given by manager. \
    Conduct interviews with people who knew the historical figure. \
    Write a narrative that captures the experiences, achievements, and personality of historical figure. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
biographer_task = Task(
    description = biographer_task_desc,
    expected_output="A comprehensive 3 paragraphs long report on the historical background of the person/subject in research",
    tools=[search_tool, webscrape_tool],
    agent=biographer,
    async_execution = True,
    context = [manager_task1]
)

In [6]:
psychologist = Agent(
    role='Psychologist',
    goal='Analyzes the personality, behavior, and psychological traits of a historical figure or someone influential',
    verbose=True,
    backstory=(
        "Holds a PhD in Psychology or Psychohistory. \
        Experienced clinical psychologist or researcher with a robust publication record in psychological journals, \
        specializing in historical figures' psychological profiles."
    ),
    llm = openAIGPT35_500,
    max_iter = 10, 
)

psychologist_task_desc = "Apply psychological theories to understand the actions and decisions of the historical figures given by manager. \ \
    Use psychohistorical methods to interpret the impact of psychological factors on historical events. \
    Provide insights into the mental and emotional state of the person/subject that is being researched. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
psychologist_task = Task(
    description = psychologist_task_desc, 
    expected_output='A comprehensive 3 paragraphs long report on the psychological analysis of the historical figures given by manager.',
    tools=[exa_tools, search_tool, webscrape_tool],
    agent=psychologist,
    async_execution = True,
    context = [manager_task1]
)

In [7]:
philosopher = Agent(
    role='Philosopher',
    goal='Analyzes the ethical and philosophical beliefs and contributions made by a historical figure.',
    verbose=True,
    backstory=(
        "PhD in Philosophy or Ethics, with a focus on ethical analysis and philosophical inquiry. \
        Published numerous scholarly articles and books on ethical and philosophical topics, \
        providing deep insights into moral considerations. "
    ),
    llm = openAIGPT35_500,
    max_iter = 10,
)

philosopher_task_desc = "Analyze moral and ethical implications of the decisions made by historical figures given by manager. \
    Study philosophical influences on the beliefs and actions made by the historical figures. \
    Provide insights into the ethical context of the time period when the historical figures lived. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
philosopher_task = Task(
    description = philosopher_task_desc,
    expected_output='A comprehensive 3 paragraphs long report and a summarized bullet points of key philosophical ideas of the historical figures given by the manager.',
    tools=[exa_tools, search_tool, webscrape_tool],
    agent=philosopher,
    async_execution = True,
    context = [manager_task1]
)

In [8]:
linguist = Agent(
    role='Linguist',
    goal='Analyzes the language, writings, and speeches of historical figures.',
    verbose=True,
    backstory=(
        "PhD in Linguistics, with extensive research on language patterns and communication styles. \
        Published numerous papers on linguistics and conducted significant fieldwork or research."
    ),
    llm = openAIGPT35_500,
)

linguist_task_desc = "Analyzes writings, speeches, and other communications by the historical figures. \
    Identify linguistic patterns and influences of the historical figure being discussed. \
    Provide context on how language shaped and was shaped by the culture of the historical figure being discussed. \
    Additionally when using tools, make sure to not read sources that exceed more than 1000 words \
    and NEVER EVER DO NOT source from https://www.wikipedia.org and any of its sub-domains."

# Research task
linguist_task = Task(
    description = linguist_task_desc,
    expected_output='A comprehensive 3 paragraphs long report on the psychological analysis of the historical figures given by the manager.',
    tools=[exa_tools, search_tool, webscrape_tool],
    agent=linguist,
    async_execution = True,
    context = [manager_task1]
)

In [9]:
manager_task2 = Task(
    description=(
        "Once you think the members of the team had some discussion about {historical_figure_1},"
        "stop the discussion but keep the memory / information that was discussed."
        "Now, the primary goal of your team is to provide a user experience where the user feels like they"
        "are talking to {historical_figure_1}."
        
        "To do so, ask and discuss with the experts / team members about what, according to their"
        "profession and knowledge what they think {historical_figure_1}'s response would be"
        "about my question '{question}'."
        
        "Then, come up with a single conclusion of what {historical_figure_1}'s response would be and "
        "write a single response while acting/role-playing as {historical_figure_1} character. "
        "The response must clearly state and show the logical steps and reasons why you (as {historical_figure_1})"
        "think that way about {question}."
        
        "An example discussion between the team members would look something like below:"
        "*** Historical Figure: {historical_figure_1} ***"
        "*** Question: Is capitalism good? ***"
        "*Biographer's opinion*"
        "I think {historical_figure_1} would answer to the question 'is capitalism good' by '...' because according to"
        "what I have researched earlier '...'"
        "*Philosopher's opinion*"
        "I think {historical_figure_1} would answer to the question 'is capitalism good' by '...' because according to"
        "his philosophical beliefs from his work '...'"
        "..."
    ),
    expected_output = (
        "A single summarized response of what the team, as a whole after discussion with each other, think"
        "{historical_figure_1} would have responded to the question '{question}'."
        "Make sure the final, single summarized response is written like an impersonation of {historical_figure_1} is talking."
    ),
    agent=manager,
    context = [manager_task1, biographer_task, psychologist_task, philosopher_task, linguist_task],
    async_execution = True,
    human_input = False
)

manager_task3 = Task(
    description=(
        "Once you think the members of the team had some discussion about {historical_figure_2},"
        "stop the discussion but keep the memory / information that was discussed."
        "Now, the primary goal of your team this time is to provide a user experience where the user feels like they"
        "are talking to {historical_figure_2} instead of {historical_figure_2} (make sure to keep both answers for {historical_figure_1} and {historical_figure_2})."
        
        "To do so, ask and discuss with the experts / team members about what, according to their"
        "profession and knowledge what they think {historical_figure_2}'s response would be"
        "about my question '{question}'."
        
        "Then, come up with a single conclusion of what {historical_figure_2}'s response would be and "
        "write a single response while acting/role-playing as {historical_figure_2} character. "
        "The response must clearly state and show the logical steps and reasons why you (as {historical_figure_2})"
        "think that way about {question}."
        
        "An example discussion between the team members would look something like below:"
        "*** Historical Figure: {historical_figure_2} ***"
        "*** Question: Is capitalism good? ***"
        "*Biographer's opinion*"
        "I think {historical_figure_2} would answer to the question 'is capitalism good' by '...' because according to"
        "what I have researched earlier '...'"
        "*Philosopher's opinion*"
        "I think {historical_figure_2} would answer to the question 'is capitalism good' by '...' because according to"
        "his philosophical beliefs from his work '...'"
        "..."
    ),
    expected_output = (
        "A single summarized response of what the team, as a whole after discussion with each other, think"
        "{historical_figure_2} would have responded to the question '{question}'."
        "Make sure the final, single summarized response is written like an impersonation of {historical_figure_2} is talking."
    ),
    agent=manager,
    context = [manager_task1, biographer_task, psychologist_task, philosopher_task, linguist_task],
    async_execution = True,
    human_input = False
)

In [10]:
# Discuss w/ time-machine
user_timemachine_discussion = Task (
    description=(
        "First show the two summarized (and impersonated) responses of {historical_figure_1} and {historical_figure_2}"
        "about {question} then wait for the user to throw a follow-up question."
        "When the user gives a follow-up question, repeat the processes done for {historical_figure_1} and {historical_figure_2}"
        "(an impersonation of a single summarized response of what the team thinks {historical_figure_1} or {historical_figure_2} thinks about the follow-up question)"
    ),
    expected_output = (
        "A single summarized response of what the team, as a whole after discussion with each other, think"
        "{historical_figure_1} and {historical_figure_2} would have responded to the follow-up questions of the user."
        
        "Based on those response, continue to debate between the impersonations of {historical_figure_1} and {historical_figure_2}"
        "if they disagree in some aspects. Make sure to state a logical reasoning why they disagree from each other."
        "If they agree, make sure to state a logical reasoning of why they agree."
       
        "Make sure the all the responses are also impersonated so that the user feels like {historical_figure_1} or {historical_figure_2} is talking."
    ),
    agent = manager,
    context = [manager_task2, manager_task3],
    human_input = True
)

In [11]:
# Instantiate your crew with a sequential process
crew = Crew(
    agents=[manager, biographer, psychologist, linguist, philosopher],
    tasks=[
        manager_task1, biographer_task, psychologist_task, philosopher_task, linguist_task, 
        manager_task2, manager_task3,
        user_timemachine_discussion
    ],
    # verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff(inputs = {
    "historical_figure_1": "Plato",
    "historical_figure_2": "Socrates",
    "question": "do you think capitalism is beneficial?",
})                                         

print("###################### FINAL ######################")
print(result)



> Entering new CrewAgentExecutor chain...
I need to delegate tasks to the subject matter experts in order to thoroughly analyze and investigate Plato and Socrates.

Action: Delegate work to coworker
Action Input: 
{
    "coworker": "Biographer",
    "task": "Summarize bibliographical information about Plato and Socrates",
    "context": "The biographer should focus on gathering and summarizing detailed information about the life, works, and impact of Plato and Socrates from various historical sources."
}


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Plato, a renowned ancient Greek philosopher, was born in Athens in 427 BC. He was a student of Socrates and went on to establish the Academy, one of the first institutions of higher learning in the Western world. Plato's most famous works include "The Republic," "Symposium," and "Phaedo," where he explores various philosophical concepts such as justice, love, and the immortality of the soul.

Soc

2024-06-23 11:28:24,282 - 6398439424 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 652, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 805, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 465,

The response from Plato, as summarized by the Philosopher, emphasizes the importance of justice, fairness, and the common good in evaluating the benefits of capitalism. Plato believes that while capitalism can lead to economic growth and innovation, it must be balanced with a commitment to ethical behavior and the well-being of all individuals in society.

Action: Delegate work to coworker
Action Input: {"coworker": "Philosopher", "task": "Summarize response about capitalism being beneficial", "context": "As Socrates, I would respond to the question of whether capitalism is beneficial by acknowledging the complexities of such a system. While capitalism may lead to economic growth, innovation, and individual freedom, it also raises ethical concerns about justice, equality, and the common good. I would emphasize the importance of self-examination, critical thinking, and the pursuit of truth in evaluating the impact of capitalism on society. Ultimately, I would encourage a thoughtful and 

2024-06-23 11:28:54,334 - 6398439424 - __init__.py-__init__:369 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 652, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/connection.py", line 805, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/hackathon/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 465,

This is the agent final answer: Based on the summarized responses of Plato and Socrates regarding whether capitalism is beneficial, both philosophers agree that capitalism has its advantages in terms of economic growth and innovation. However, they also share concerns about the ethical implications of capitalism, particularly related to justice, equality, and the common good. While Plato emphasizes the importance of promoting ethical behavior and fairness in wealth distribution, Socrates focuses on critical thinking and self-examination to address the ethical concerns raised by capitalism. Their agreement stems from a shared belief in the need for a thoughtful and nuanced discussion on the impact of capitalism on society, considering both its benefits and drawbacks in promoting human flourishing and moral integrity.
Please provide a feedback:  what are some of the concerns they think about such economical system? Dive more in detail why they think that way.


Action: Ask question to coworker
Action Input: {"coworker": "Philosopher", "question": "What are some of the concerns Plato and Socrates have about the ethical implications of capitalism, and why do they think that way?", "context": "As Plato, I believe that the question of whether capitalism is beneficial is a complex one that requires careful consideration. While capitalism can stimulate economic growth and innovation, it also has the potential to exacerbate inequality and exploitation. As a philosopher who values justice and the pursuit of knowledge, I would argue that a just society should prioritize the well-being of all individuals and ensure that wealth is distributed fairly. In my view, a system that promotes the common good and ethical behavior is essential for creating a harmonious and virtuous society. Therefore, while capitalism may have its advantages, it is crucial to temper its effects with a commitment to justice, reason, and the greater good of society. As Socrates, I 